# TEST DELLE SOLUZIONI

In [1]:
import numpy as np
import torch
import cvxpy as cp

def knapsack_specialized_pruning(xi, v, w, C):
    b_list = []
    b = 0

    # Compute breakpoint vector x_plus
    while True:
        delta_xi = (xi[b + 1:] - xi[b])
        delta_v = (v[b + 1:] - v[b])
        b = torch.argmin(delta_xi / delta_v) + 1 + b_list[-1] if b_list else 0

        if b != C - 1:
            b_list.append(int(b))

        if b + 1 > C - 1:
            break
    b_list.append(C - 1)
    x_plus = torch.zeros(C, dtype=torch.int32)
    b_tensor = torch.tensor(b_list, dtype=torch.int32)
    x_plus[b_tensor] = 1

    if(w > v[-1] or w < v[0]):
        #print("No solutions.")
        x = None
        objective_values = None
    else:
        b_vector = np.argsort(xi/v)
        b = 0
        b_list_new = [b_vector[b]] 
        while(w/v[b_vector[b]] > 1 or x_plus[b_vector[b]] == 0 or int(b_vector[b]) < int(b_list_new[0])):
            b += 1
            if(x_plus[b_vector[b]] == 1 and int(b_vector[b]) > int(b_list_new[0])):
                b_list_new.append(b_vector[b])
            if(len(b_list_new) > 2):
                b_list_new.pop(0)
        
        x = np.zeros(C)
        
        if(len(b_list_new) == 1):
            x[b_list_new[0]] = round(float(w/v[b_list_new[0]]), 2)
        else:
            x1, x2 = torch.zeros(2, len(w), C, dtype=torch.float32)
            x1[torch.arange(len(w)), b_list_new[0]] = 1
            x2[torch.arange(len(w)), b_list_new[1]] = 1
            numerator = w - torch.matmul(x2, v)
            denominator = torch.matmul((x1 - x2), v)
            theta = numerator / denominator
            x[b_list_new[0]] = round(float(theta), 2)
            x[b_list_new[1]] = round(float(1 - theta), 2)
        objective_values = torch.matmul(torch.from_numpy(x).float(), xi) 
    
    lambda_opt = 0
    
    return x, lambda_opt, objective_values, x_plus

# Dimension of vectors xi, v, and x
C = 5
M = 1

np.random.seed(43)  
torch.manual_seed(43)
for i in range(1000):
    xi = torch.sort(torch.rand(C))[0]
    v = torch.linspace(0, 1 - (1 / C), C)
    w = torch.rand(M)
    #w = v[torch.randint(0, C, (1,))]  
    
    #print(f"\nIteration: {i}")
    #print("DATI:")
    #print("xi:", np.array2string(np.array(xi), precision=2, separator=', '))
    #print("v:", np.array2string(np.array(v), precision=2, separator=', '))
    #print("w:", w, "\n")
    
    x_opt, lambda_opt, phi_opt, x_plus = knapsack_specialized_pruning(xi, v, w, C)
    
    # cvxpy solution with inequality constraint (pruning)
    x_pruning = cp.Variable(C)
    equality_constraint_pruning = v @ x_pruning == w
    constraints_pruning = [
        equality_constraint_pruning,
        cp.sum(x_pruning) <= 1,
        x_pruning >= 0,
        x_pruning <= 1
    ]
    objective_pruning = cp.Minimize(xi @ x_pruning)
    prob_pruning = cp.Problem(objective_pruning, constraints_pruning)
    prob_pruning.solve(solver=cp.CLARABEL)

    #if prob_pruning.value is not None and x_pruning.value is not None:
    #    print(f"Objective value (pruning): {prob_pruning.value:.2f}")
    #    print("Optimal x (pruning):", 
    #          np.array2string(x_pruning.value, precision=2, separator=', ', suppress_small=True))
    #else:
    #    print("No solution found for the problem with pruning.")
        
    if(phi_opt is not None):
        #print("Objective value (knapsack_specialized_pruning):", round(float(phi_opt), 2))
        #print("Optimal x (knapsack_specialized_pruning):", x_opt)
        x1 = []
        x2 = []
        SUCCESS = 1
        for j in range(C):
            x1.append(round(float(x_pruning.value[j]), 2))
            x2.append(round(float(x_opt[j]), 2))
        if(x1 != x2):
            SUCCESS = 0
            print("xi:", np.array2string(np.array(xi), precision=2, separator=', '))
            print("v:", np.array2string(np.array(v), precision=2, separator=', '))
            print("w:", round(float(w), 4), "\n")
            print("xi/v:", xi/v)
            for k in range(C):
                print(f"w/v[{k}]: {w/v[k]}")
            print("x_plus:", x_plus)
            print("Soluzione del solver:", x1)
            print("Soluzione mia:", x2)
            print("f.o. solver:", round(prob_pruning.value, 4))
            print("f.o. mia:", round(float(phi_opt), 4))
            print(f"Il vincolo w=v*x è soddisfatto con la mia soluzione? w = {w}, v*x={v @ x_opt}")
            print("-"*40)
            
    #else:
    #    print("Objective value (knapsack_specialized_pruning):", phi_opt)
    #    print("Optimal x (knapsack_specialized_pruning):", x_opt)
    
    #print("-"*40)
if(SUCCESS == 1):
    print("TEST CONCLUSO CON SUCCESSO!")

TEST CONCLUSO CON SUCCESSO!


# Calcolo dei moltiplicatori: caso con due vincoli attivi

In [2]:
# Funzione per formattare e stampare un array
def print_dual(label, values):
    formatted = np.where(np.abs(values) < epsilon, 0.0, values)
    print(f"{label}: {np.array2string(formatted, precision=2, separator=', ', suppress_small=True)}")
    
np.random.seed(67)  # Con 45 ho fatto l'esempio sull'Ipad
torch.manual_seed(67)

# Soglia sotto cui i numeri vengono considerati come zero
epsilon = 1e-5

C = 5

for _ in range(50):
    xi = torch.sort(torch.rand(C))[0]
    v = torch.linspace(0, 1 - (1 / C), C)
    w = torch.rand(M)
    #w = v[torch.randint(0, C, (1,))]  

    x_opt, lambda_opt, phi_opt, x_plus = knapsack_specialized_pruning(xi, v, w, C)
    
    if(x_opt is None):
        continue

    if(np.count_nonzero(x_opt) == 2):
    #if(x_opt[2] != 0 and x_opt[3] != 0):
        print("DATI:")
        print("xi:", np.array2string(np.array(xi), precision=2, separator=', '))
        print("v:", np.array2string(np.array(v), precision=2, separator=', '))
        print("w:", w, "\n")

        print("x_opt:", x_opt)
        print("f.o.:", phi_opt)
        print("lambda_opt: ?")

        x_pruning = cp.Variable(C)
        equality_constraint_pruning = v @ x_pruning == w
        constraints_pruning = [
            equality_constraint_pruning,
            cp.sum(x_pruning) <= 1,
            x_pruning >= 0,
            x_pruning <= 1
        ]
        objective_pruning = cp.Minimize(xi @ x_pruning)
        prob_pruning = cp.Problem(objective_pruning, constraints_pruning)
        prob_pruning.solve(solver=cp.CLARABEL)

        if prob_pruning.value is not None and x_pruning.value is not None:
            print("\n---------------------- SOLVER SOLUTION -------------------------")
            print(f"Objective value (pruning): {prob_pruning.value:.2f}")
            print("Optimal x (pruning):", 
                  np.array2string(x_pruning.value, precision=2, separator=', ', suppress_small=True))
            # Stampa dei moltiplicatori ottimali
            print_dual("Dual variable for equality constraint (v @ x == w)", equality_constraint_pruning.dual_value)
            print_dual("Dual variable for sum(x) <= 1", np.array([constraints_pruning[1].dual_value]))
            print_dual("Dual variable for x >= 0", np.array(constraints_pruning[2].dual_value))
            print_dual("Dual variable for x <= 1", np.array(constraints_pruning[3].dual_value))
        else:
            print("No solution found for the problem with pruning.")

        # ---------------------------------------------------------------------------------------------------
        # ---------------------------------------------------------------------------------------------------
        # ---------------------------------------------------------------------------------------------------

        print("\n-------------------- METODO SPECIALIZZATO ---------------------")

        matrice = np.zeros((C, C))

        matrice[0] = v
        matrice[1] = np.ones(C, dtype=int)

        zero_indices = np.where(x_opt == 0)[0]

        for i in range(C-2):
            if i < len(zero_indices):
                matrice[i + 2][zero_indices[i]] = -1

        det = np.linalg.det(matrice)
        if det != 0:
            inversa = np.linalg.inv(matrice)
        else:
            print("La matrice non è invertibile (determinante nullo).")

        if(float((xi @ inversa)[0]) > 0):
            matrice = np.zeros((C, C))

            matrice[0] = -v
            matrice[1] = np.ones(C, dtype=int)

            zero_indices = np.where(x_opt == 0)[0]

            for i in range(C-2):
                if i < len(zero_indices):
                    matrice[i + 2][zero_indices[i]] = -1

            det = np.linalg.det(matrice)
            if det != 0:
                inversa = np.linalg.inv(matrice)
            else:
                print("La matrice non è invertibile (determinante nullo).")

        print("La matrice di base è:\n", matrice)
        print("L'inversa della matrice è:\n", inversa)
        if(x_opt is not None):
            print("\n-------------------------- RISULTATI --------------------------")
            print("Dual variable for equality constraint (v @ x == w):", float((xi @ inversa)[0]))
            print("VS")
            print("Dual variable for equality constraint (v @ x == w):", equality_constraint_pruning.dual_value[0])

            x_star = inversa @ np.array([-float(w), 1, 0, 0, 0])
            print("x_star:", np.array2string(np.array(x_star), precision=2, separator=', '))
            print("VS")
            print("x_star:", np.array2string(x_pruning.value, precision=2, separator=', ', suppress_small=True))

            prima_colonna = inversa[:, 0]
            non_nulli = prima_colonna[prima_colonna != 0]
            print("Elementi non nulli della prima colonna:", non_nulli)

        print("\n"*20)

DATI:
xi: [0.08, 0.09, 0.37, 0.5 , 0.84]
v: [0. , 0.2, 0.4, 0.6, 0.8]
w: tensor([0.3855]) 

x_opt: [0.   0.54 0.   0.46 0.  ]
f.o.: tensor(0.2783)
lambda_opt: ?

---------------------- SOLVER SOLUTION -------------------------
Objective value (pruning): 0.28
Optimal x (pruning): [0.  , 0.54, 0.  , 0.46, 0.  ]
Dual variable for equality constraint (v @ x == w): [-1.01]
Dual variable for sum(x) <= 1: [0.11]
Dual variable for x >= 0: [0.19, 0.  , 0.08, 0.  , 0.15]
Dual variable for x <= 1: [0., 0., 0., 0., 0.]

-------------------- METODO SPECIALIZZATO ---------------------
La matrice di base è:
 [[-0.         -0.2        -0.40000001 -0.60000002 -0.80000001]
 [ 1.          1.          1.          1.          1.        ]
 [-1.          0.          0.          0.          0.        ]
 [ 0.          0.         -1.          0.          0.        ]
 [ 0.          0.          0.          0.         -1.        ]]
L'inversa della matrice è:
 [[ 0.          0.         -1.          0.          0.  

# Caso con un vincolo attivo (diverso da 1)

In [3]:
# Funzione per formattare e stampare un array
def print_dual(label, values):
    formatted = np.where(np.abs(values) < epsilon, 0.0, values)
    print(f"{label}: {np.array2string(formatted, precision=2, separator=', ', suppress_small=True)}")
    
np.random.seed(63)  # Con 45 ho fatto l'esempio sull'Ipad
torch.manual_seed(63)

# Soglia sotto cui i numeri vengono considerati come zero
epsilon = 1e-5

C = 5
for i in range(50):

    xi = torch.sort(torch.rand(C))[0]
    v = torch.linspace(0, 1 - (1 / C), C)
    w = torch.rand(M)
    #w = v[torch.randint(0, C, (1,))]  

    x_opt, lambda_opt, phi_opt, x_plus = knapsack_specialized_pruning(xi, v, w, C)
    if(x_opt is None):
        continue

    if(np.count_nonzero(x_opt) == 1):
        if(x_opt[1] != 0):
            print("DATI:")
            print("xi:", np.array2string(np.array(xi), precision=2, separator=', '))
            print("v:", np.array2string(np.array(v), precision=2, separator=', '))
            print("w:", w, "\n")

            print("x_opt:", x_opt)
            print("f.o.:", phi_opt)
            print("lambda_opt: ?")

            x_pruning = cp.Variable(C)
            equality_constraint_pruning = v @ x_pruning == w
            constraints_pruning = [
                equality_constraint_pruning,
                cp.sum(x_pruning) <= 1,
                x_pruning >= 0,
                x_pruning <= 1
            ]
            objective_pruning = cp.Minimize(xi @ x_pruning)
            prob_pruning = cp.Problem(objective_pruning, constraints_pruning)
            prob_pruning.solve(solver=cp.CLARABEL)

            if prob_pruning.value is not None and x_pruning.value is not None:
                print("\n---------------------- SOLVER SOLUTION -------------------------")
                print(f"Objective value (pruning): {prob_pruning.value:.2f}")
                print("Optimal x (pruning):", 
                      np.array2string(x_pruning.value, precision=2, separator=', ', suppress_small=True))
                # Stampa dei moltiplicatori ottimali
                print_dual("Dual variable for equality constraint (v @ x == w)", equality_constraint_pruning.dual_value)
                print_dual("Dual variable for sum(x) <= 1", np.array([constraints_pruning[1].dual_value]))
                print_dual("Dual variable for x >= 0", np.array(constraints_pruning[2].dual_value))
                print_dual("Dual variable for x <= 1", np.array(constraints_pruning[3].dual_value))
            else:
                print("No solution found for the problem with pruning.")

            # ---------------------------------------------------------------------------------------------------
            # ---------------------------------------------------------------------------------------------------
            # ---------------------------------------------------------------------------------------------------

            print("\n-------------------- METODO SPECIALIZZATO ---------------------")

            # Calcolo la matrice
            matrice = np.zeros((C, C))
            matrice[0] = -v
            matrice[1] = np.array([-1 if i == 0 else 0 for i in range(C)])
            matrice[2] = np.array([-1 if i == 2 else 0 for i in range(C)])
            matrice[3] = np.array([-1 if i == 3 else 0 for i in range(C)])
            matrice[4] = np.array([-1 if i == 4 else 0 for i in range(C)])

            # Calcolo l'inversa
            det = np.linalg.det(matrice)
            if det != 0:
                inversa = np.linalg.inv(matrice)
            else:
                print("La matrice non è invertibile (determinante nullo).")

            if ((xi @ inversa) > 0).all():
                # Calcolo la matrice
                matrice = np.zeros((C, C))
                matrice[0] = v
                matrice[1] = np.array([-1 if i == 0 else 0 for i in range(C)])
                matrice[2] = np.array([-1 if i == 2 else 0 for i in range(C)])
                matrice[3] = np.array([-1 if i == 3 else 0 for i in range(C)])
                matrice[4] = np.array([-1 if i == 4 else 0 for i in range(C)])

                # Calcolo l'inversa
                det = np.linalg.det(matrice)
                if det != 0:
                    inversa = np.linalg.inv(matrice)
                else:
                    print("La matrice non è invertibile (determinante nullo).")

            print("xi @ inversa:", xi @ inversa)

            # Cambio base
            if(float((xi @ inversa)[0]) > 0):
                matrice = np.zeros((C, C))
                matrice[0] = -v
                matrice[1] = np.ones(C, dtype=int)
                zero_indices = np.where(x_opt == 0)[0]
                for i in range(C-2):
                    if i < len(zero_indices):
                        matrice[i + 2][zero_indices[i]] = -1
                det = np.linalg.det(matrice)
                if det != 0:
                    inversa = np.linalg.inv(matrice)
                else:
                    print("La matrice non è invertibile (determinante nullo).")

            # Stampo i risultati
            print("La matrice di base è:\n", matrice)
            print("L'inversa della matrice è:\n", inversa)
            if(x_opt is not None):
                print("\n-------------------------- RISULTATI --------------------------")
                print("Dual variable for equality constraint (v @ x == w):", float((xi @ inversa)[0]))
                print("VS")
                print("Dual variable for equality constraint (v @ x == w):", equality_constraint_pruning.dual_value[0])

                x_star = inversa @ np.array([-float(w), 1, 0, 0, 0])
                print("x_star:", np.array2string(np.array(x_star), precision=2, separator=', '))
                print("VS")
                print("x_star:", np.array2string(x_pruning.value, precision=2, separator=', ', suppress_small=True))

                prima_colonna = inversa[:, 0]
                non_nulli = prima_colonna[prima_colonna != 0]
                print("Elementi non nulli della prima colonna:", non_nulli)

            print("\n"*20)

DATI:
xi: [0.08, 0.24, 0.56, 0.95, 1.  ]
v: [0. , 0.2, 0.4, 0.6, 0.8]
w: tensor([0.0262]) 

x_opt: [0.   0.13 0.   0.   0.  ]
f.o.: tensor(0.0312)
lambda_opt: ?

---------------------- SOLVER SOLUTION -------------------------
Objective value (pruning): 0.03
Optimal x (pruning): [0.  , 0.13, 0.  , 0.  , 0.  ]
Dual variable for equality constraint (v @ x == w): [-1.2]
Dual variable for sum(x) <= 1: [0.]
Dual variable for x >= 0: [0.08, 0.  , 0.08, 0.23, 0.04]
Dual variable for x <= 1: [0., 0., 0., 0., 0.]

-------------------- METODO SPECIALIZZATO ---------------------
xi @ inversa: tensor([-1.1997, -0.0842, -0.0756, -0.2272, -0.0386], dtype=torch.float64)
La matrice di base è:
 [[-0.         -0.2        -0.40000001 -0.60000002 -0.80000001]
 [-1.          0.          0.          0.          0.        ]
 [ 0.          0.         -1.          0.          0.        ]
 [ 0.          0.          0.         -1.          0.        ]
 [ 0.          0.          0.          0.         -1.        

# Caso con un vincolo attivo (uguale a 1)

In [4]:
# Funzione per formattare e stampare un array
def print_dual(label, values):
    formatted = np.where(np.abs(values) < epsilon, 0.0, values)
    print(f"{label}: {np.array2string(formatted, precision=2, separator=', ', suppress_small=True)}")
    
np.random.seed(63)  # Con 45 ho fatto l'esempio sull'Ipad
torch.manual_seed(63)

# Soglia sotto cui i numeri vengono considerati come zero
epsilon = 1e-5

C = 5
for i in range(4000):

    xi = torch.sort(torch.rand(C))[0]
    v = torch.linspace(0, 1 - (1 / C), C)
    w = torch.rand(M)
    #w = v[torch.randint(0, C, (1,))]  

    x_opt, lambda_opt, phi_opt, x_plus = knapsack_specialized_pruning(xi, v, w, C)
    if(x_opt is None):
        continue

    if(np.count_nonzero(x_opt) == 1):
        if(np.any(x_opt == 1)):
            print("DATI:")
            print("xi:", np.array2string(np.array(xi), precision=2, separator=', '))
            print("v:", np.array2string(np.array(v), precision=2, separator=', '))
            print("w:", w, "\n")

            print("x_opt:", x_opt)
            print("f.o.:", phi_opt)
            print("lambda_opt: ?")

            x_pruning = cp.Variable(C)
            equality_constraint_pruning = v @ x_pruning == w
            constraints_pruning = [
                equality_constraint_pruning,
                cp.sum(x_pruning) <= 1,
                x_pruning >= 0,
                x_pruning <= 1
            ]
            objective_pruning = cp.Minimize(xi @ x_pruning)
            prob_pruning = cp.Problem(objective_pruning, constraints_pruning)
            prob_pruning.solve(solver=cp.CLARABEL)

            if prob_pruning.value is not None and x_pruning.value is not None:
                print("\n---------------------- SOLVER SOLUTION -------------------------")
                print(f"Objective value (pruning): {prob_pruning.value:.2f}")
                print("Optimal x (pruning):", 
                      np.array2string(x_pruning.value, precision=2, separator=', ', suppress_small=True))
                # Stampa dei moltiplicatori ottimali
                print_dual("Dual variable for equality constraint (v @ x == w)", equality_constraint_pruning.dual_value)
                print_dual("Dual variable for sum(x) <= 1", np.array([constraints_pruning[1].dual_value]))
                print_dual("Dual variable for x >= 0", np.array(constraints_pruning[2].dual_value))
                print_dual("Dual variable for x <= 1", np.array(constraints_pruning[3].dual_value))
            else:
                print("No solution found for the problem with pruning.")

            # ---------------------------------------------------------------------------------------------------
            # ---------------------------------------------------------------------------------------------------
            # ---------------------------------------------------------------------------------------------------

            print("\n-------------------- METODO SPECIALIZZATO ---------------------")

            # Calcolo la matrice
            matrice = np.zeros((C, C))
            matrice[0] = -v
            matrice[1] = np.ones(C, dtype=int)
            matrice[2] = np.array([-1 if i == 0 else 0 for i in range(C)])
            matrice[3] = np.array([-1 if i == 2 else 0 for i in range(C)])
            matrice[4] = np.array([-1 if i == 3 else 0 for i in range(C)])

            # Calcolo l'inversa
            det = np.linalg.det(matrice)
            if det != 0:
                inversa = np.linalg.inv(matrice)
            else:
                print("La matrice non è invertibile (determinante nullo).")

            if ((xi @ inversa) > 0).all():
                # Calcolo la matrice
                matrice = np.zeros((C, C))
                matrice[0] = v
                matrice[1] = np.ones(C, dtype=int)
                matrice[2] = np.array([-1 if i == 0 else 0 for i in range(C)])
                matrice[3] = np.array([-1 if i == 2 else 0 for i in range(C)])
                matrice[4] = np.array([-1 if i == 3 else 0 for i in range(C)])

                # Calcolo l'inversa
                det = np.linalg.det(matrice)
                if det != 0:
                    inversa = np.linalg.inv(matrice)
                else:
                    print("La matrice non è invertibile (determinante nullo).")

            print("xi @ inversa:", xi @ inversa)

            # Cambio base
            if(float((xi @ inversa)[0]) > 0):
                matrice = np.zeros((C, C))
                matrice[0] = -v
                matrice[1] = np.ones(C, dtype=int)
                zero_indices = np.where(x_opt == 0)[0]
                for i in range(C-2):
                    if i < len(zero_indices):
                        matrice[i + 2][zero_indices[i]] = -1
                det = np.linalg.det(matrice)
                if det != 0:
                    inversa = np.linalg.inv(matrice)
                else:
                    print("La matrice non è invertibile (determinante nullo).")

            # Stampo i risultati
            print("La matrice di base è:\n", matrice)
            print("L'inversa della matrice è:\n", inversa)
            if(x_opt is not None):
                print("\n-------------------------- RISULTATI --------------------------")
                print("Dual variable for equality constraint (v @ x == w):", float((xi @ inversa)[0]))
                print("VS")
                print("Dual variable for equality constraint (v @ x == w):", equality_constraint_pruning.dual_value[0])

                x_star = inversa @ np.array([-float(w), 1, 0, 0, 0])
                print("x_star:", np.array2string(np.array(x_star), precision=2, separator=', '))
                print("VS")
                print("x_star:", np.array2string(x_pruning.value, precision=2, separator=', ', suppress_small=True))

                prima_colonna = inversa[:, 0]
                non_nulli = prima_colonna[prima_colonna != 0]
                print("Elementi non nulli della prima colonna:", non_nulli)

            print("\n"*20)

DATI:
xi: [0.  , 0.13, 0.68, 0.68, 0.98]
v: [0. , 0.2, 0.4, 0.6, 0.8]
w: tensor([0.1992]) 

x_opt: [0. 1. 0. 0. 0.]
f.o.: tensor(0.1284)
lambda_opt: ?

---------------------- SOLVER SOLUTION -------------------------
Objective value (pruning): 0.13
Optimal x (pruning): [0., 1., 0., 0., 0.]
Dual variable for equality constraint (v @ x == w): [-0.64]
Dual variable for sum(x) <= 1: [0.]
Dual variable for x >= 0: [0.  , 0.  , 0.42, 0.29, 0.47]
Dual variable for x <= 1: [0., 0., 0., 0., 0.]

-------------------- METODO SPECIALIZZATO ---------------------
xi @ inversa: tensor([-1.4187, -0.1553, -0.1582, -0.2649,  0.0180], dtype=torch.float64)
La matrice di base è:
 [[-0.         -0.2        -0.40000001 -0.60000002 -0.80000001]
 [ 1.          1.          1.          1.          1.        ]
 [-1.          0.          0.          0.          0.        ]
 [ 0.          0.         -1.          0.          0.        ]
 [ 0.          0.          0.         -1.          0.        ]]
L'inversa della 

DATI:
xi: [0.01, 0.01, 0.31, 0.69, 0.91]
v: [0. , 0.2, 0.4, 0.6, 0.8]
w: tensor([0.2016]) 

x_opt: [0. 1. 0. 0. 0.]
f.o.: tensor(0.0083)
lambda_opt: ?

---------------------- SOLVER SOLUTION -------------------------
Objective value (pruning): 0.01
Optimal x (pruning): [0., 1., 0., 0., 0.]
Dual variable for equality constraint (v @ x == w): [-1.5]
Dual variable for sum(x) <= 1: [0.29]
Dual variable for x >= 0: [0.3 , 0.  , 0.  , 0.08, 0.  ]
Dual variable for x <= 1: [0., 0., 0., 0., 0.]

-------------------- METODO SPECIALIZZATO ---------------------
xi @ inversa: tensor([-1.5049e+00, -2.9267e-01, -3.0002e-01, -5.2780e-04, -7.7748e-02],
       dtype=torch.float64)
La matrice di base è:
 [[-0.         -0.2        -0.40000001 -0.60000002 -0.80000001]
 [ 1.          1.          1.          1.          1.        ]
 [-1.          0.          0.          0.          0.        ]
 [ 0.          0.         -1.          0.          0.        ]
 [ 0.          0.          0.         -1.          0

DATI:
xi: [0.14, 0.6 , 0.62, 0.69, 0.85]
v: [0. , 0.2, 0.4, 0.6, 0.8]
w: tensor([0.7994]) 

x_opt: [0. 0. 0. 0. 1.]
f.o.: tensor(0.8522)
lambda_opt: ?

---------------------- SOLVER SOLUTION -------------------------
Objective value (pruning): 0.85
Optimal x (pruning): [-0.,  0.,  0.,  0.,  1.]
Dual variable for equality constraint (v @ x == w): [-1.07]
Dual variable for sum(x) <= 1: [0.]
Dual variable for x >= 0: [0.14, 0.39, 0.2 , 0.05, 0.  ]
Dual variable for x <= 1: [0., 0., 0., 0., 0.]

-------------------- METODO SPECIALIZZATO ---------------------
xi @ inversa: tensor([-0.4171,  0.5186,  0.3803,  0.0638,  0.0792], dtype=torch.float64)
La matrice di base è:
 [[-0.         -0.2        -0.40000001 -0.60000002 -0.80000001]
 [ 1.          1.          1.          1.          1.        ]
 [-1.          0.          0.          0.          0.        ]
 [ 0.          0.         -1.          0.          0.        ]
 [ 0.          0.          0.         -1.          0.        ]]
L'inversa d

xi @ inversa: tensor([-0.7397,  0.0275, -0.0941, -0.1440, -0.0216], dtype=torch.float64)
La matrice di base è:
 [[-0.         -0.2        -0.40000001 -0.60000002 -0.80000001]
 [ 1.          1.          1.          1.          1.        ]
 [-1.          0.          0.          0.          0.        ]
 [ 0.          0.         -1.          0.          0.        ]
 [ 0.          0.          0.         -1.          0.        ]]
L'inversa della matrice è:
 [[ 0.          0.         -1.          0.          0.        ]
 [ 1.66666664  1.33333333  1.33333333  0.66666667  0.33333331]
 [-0.         -0.         -0.         -1.         -0.        ]
 [-0.         -0.         -0.         -0.         -1.        ]
 [-1.66666664 -0.33333333 -0.33333333  0.33333333  0.66666669]]

-------------------------- RISULTATI --------------------------
Dual variable for equality constraint (v @ x == w): -0.7397021739725942
VS
Dual variable for equality constraint (v @ x == w): -0.7741231367701552
x_star: [0., 0.,

# TEST DEI MOLTIPLICATORI (SENZA IL VINCOLO ATTIVO = 1)

In [5]:
# Dimension of vectors xi, v, and x
C = 32
M = 1

SUCCESS = 1

np.random.seed(43)  
torch.manual_seed(43)
for _ in range(10000):
    xi = torch.sort(torch.rand(C))[0]
    v = torch.linspace(0, 1 - (1 / C), C)
    w = torch.rand(M)
    #w = v[torch.randint(0, C, (1,))]  
    
    # cvxpy solution with inequality constraint (pruning)
    x_pruning = cp.Variable(C)
    equality_constraint_pruning = v @ x_pruning == w
    constraints_pruning = [
        equality_constraint_pruning,
        cp.sum(x_pruning) <= 1,
        x_pruning >= 0,
        x_pruning <= 1
    ]
    objective_pruning = cp.Minimize(xi @ x_pruning)
    prob_pruning = cp.Problem(objective_pruning, constraints_pruning)
    prob_pruning.solve(solver=cp.ECOS)
    
    if(equality_constraint_pruning.dual_value is not None):
        y1 = round(equality_constraint_pruning.dual_value[0], 2)
        #print("Lagrange multiplier:", y1)
        #print("np.count_nonzero(np.abs(x_pruning.value) > 1e-6):", np.count_nonzero(np.abs(x_pruning.value) > 1e-6))
        if(np.count_nonzero(np.abs(x_pruning.value) > 1e-6) == 2):
            i = np.nonzero(np.abs(x_pruning.value) > 1e-6) # i[0] è la lista [indice1, indice2]
            diff = i[0][1] - i[0][0]
            y2 = round(float(-C / diff * (xi[i[0][1]] - xi[i[0][0]])), 2)
        else:
            i = np.nonzero(np.abs(x_pruning.value) > 1e-6)
            y2 = round(float(-C / i[0][0] * xi[i[0][0]]), 2) # i[0][0] è l'indice
        if(y1 != y2 and np.all(x_pruning.value != 1)):
            print("C'È STATO UN ERRORE NEL CALCOLO DEI MOLTIPLICATORI!\n")
            print("DATI:")
            print("xi:", np.array2string(np.array(xi), precision=2, separator=', '))
            print("v:", np.array2string(np.array(v), precision=2, separator=', '))
            print("w:", w, "\n")
            print("y1:", y1)
            print("y2:", y2)
            SUCCESS = 0
        #print("Lagrange multiplier:", y2, "\n")
if(SUCCESS == 1):
    print("TEST EFFETTUATO CON SUCCESSO!")

/opt/anaconda3/lib/python3.11/site-packages/cvxpy/reductions/solvers/solving_chain.py:356: FutureWarning: 
    You specified your problem should be solved by ECOS. Starting in
    CXVPY 1.6.0, ECOS will no longer be installed by default with CVXPY.
    Please either add ECOS as an explicit install dependency to your project
    or switch to our new default solver, Clarabel, by either not specifying a
    solver argument or specifying ``solver=cp.CLARABEL``. To suppress this
    warning while continuing to use ECOS, you can filter this warning using
    Python's ``warnings`` module until you are using 1.6.0.
    
  warnings.warn(ECOS_DEP_DEPRECATION_MSG, FutureWarning)


TEST EFFETTUATO CON SUCCESSO!


# TEST DEI MOLTIPLICATORI (CON IL VINCOLO ATTIVO = 1)

### In realtà questo test è uguale a quello della cella sopra. 
### Ho solo sostituito la condizione 
### if(y1 != y2 and np.all(x_pruning.value != 1)): 
### con la condizione
### if(y1 != y2):

In [6]:
# Dimension of vectors xi, v, and x
C = 5
M = 1

SUCCESS = 1

np.random.seed(45)  
torch.manual_seed(45)
for _ in range(10000):
    xi = torch.sort(torch.rand(C))[0]
    v = torch.linspace(0, 1 - (1 / C), C)
    w = torch.rand(M)
    #w = v[torch.randint(0, C, (1,))]  
    
    # cvxpy solution with inequality constraint (pruning)
    x_pruning = cp.Variable(C)
    equality_constraint_pruning = v @ x_pruning == w
    constraints_pruning = [
        equality_constraint_pruning,
        cp.sum(x_pruning) <= 1,
        x_pruning >= 0,
        x_pruning <= 1
    ]
    objective_pruning = cp.Minimize(xi @ x_pruning)
    prob_pruning = cp.Problem(objective_pruning, constraints_pruning)
    prob_pruning.solve(solver=cp.ECOS)
    if(equality_constraint_pruning.dual_value is not None):
        y1 = round(equality_constraint_pruning.dual_value[0], 2)
        #print("Lagrange multiplier:", y1)
        #print("np.count_nonzero(np.abs(x_pruning.value) > 1e-6):", np.count_nonzero(np.abs(x_pruning.value) > 1e-6))
        if(np.count_nonzero(np.abs(x_pruning.value) > 1e-6) == 2):
            i = np.nonzero(np.abs(x_pruning.value) > 1e-6) # i[0] è la lista [indice1, indice2]
            diff = i[0][1] - i[0][0]
            y2 = round(float(-C / diff * (xi[i[0][1]] - xi[i[0][0]])), 2)
        else:
            i = np.nonzero(np.abs(x_pruning.value) > 1e-6)
            y2 = round(-float(xi[i[0][0]] / v[i[0][0]]), 2) # i[0][0] è l'indice
        if(y1 != y2):
            print("C'È STATO UN ERRORE NEL CALCOLO DEI MOLTIPLICATORI!\n")
            print("DATI:")
            print("xi:", np.array2string(np.array(xi), precision=2, separator=', '))
            print("v:", np.array2string(np.array(v), precision=2, separator=', '))
            print("w:", w, "\n")
            print("y1:", y1)
            print("y2:", y2)
            SUCCESS = 0
        #print("Lagrange multiplier:", y2, "\n")
if(SUCCESS == 1):
    print("TEST EFFETTUATO CON SUCCESSO!")

TEST EFFETTUATO CON SUCCESSO!


# TEST DELLE SOLUZIONI E DEI MOLTIPLICATORI CON v GENERCIO

In [7]:
def knapsack_specialized_pruning_complete(xi, v, w, C):
    b_list = []
    b = 0

    # Compute breakpoint vector x_plus
    while True:
        delta_xi = (xi[b + 1:] - xi[b])
        delta_v = (v[b + 1:] - v[b])
        b = torch.argmin(delta_xi / delta_v) + 1 + b_list[-1] if b_list else 0

        if b != C - 1:
            b_list.append(int(b))

        if b + 1 > C - 1:
            break
    b_list.append(C - 1)
    x_plus = torch.zeros(C, dtype=torch.int32)
    b_tensor = torch.tensor(b_list, dtype=torch.int32)
    x_plus[b_tensor] = 1

    if(w > v[-1] or w < v[0]):
        #print("No solutions.")
        x = None
        objective_values = None
    else:
        b_vector = np.argsort(xi/v)
        b = np.searchsorted(xi/v, 0)
        b_list_new = [b_vector[b]] 
        #print(f"Inizializzazione di b_vector[b]: {b_vector[b]}")
        #print(f"w/v[b_vector[b]] > 1: {w/v[b_vector[b]] > 1}, x_plus[b_vector[b]] == 0: {x_plus[b_vector[b]] == 0}, int(b_vector[b]) < int(b_list_new[0]): {int(b_vector[b]) < int(b_list_new[0])}")
        while(w/v[b_vector[b]] > 1 or x_plus[b_vector[b]] == 0 or int(b_vector[b]) < int(b_list_new[0])):
            b += 1
            if(v[b_vector[b]] < 0):
                continue
            if(x_plus[b_vector[b]] == 1 and int(b_vector[b]) > int(b_list_new[0])):
                b_list_new.append(b_vector[b])
                #print(f"w: {w}, v[b_vector[b]]: {v[b_vector[b]]}, w/v[b_vector[b]]: {w/v[b_vector[b]]}")
                #print(f"b_list_new:", b_list_new)
                #print("xi (sicurezza):", xi)
            if(len(b_list_new) > 2):
                b_list_new.pop(0)
        #print(f"b_list_new:", b_list_new)
        x = np.zeros(C)
        
        if(len(b_list_new) == 1):
            x[b_list_new[0]] = round(float(w/v[b_list_new[0]]), 2)
        else:
            x1, x2 = torch.zeros(2, len(w), C, dtype=torch.float32)
            x1[torch.arange(len(w)), b_list_new[0]] = 1
            x2[torch.arange(len(w)), b_list_new[1]] = 1
            numerator = w - torch.matmul(x2, v)
            denominator = torch.matmul((x1 - x2), v)
            theta = numerator / denominator
            x[b_list_new[0]] = round(float(theta), 5)
            x[b_list_new[1]] = round(float(1 - theta), 5)
        objective_values = torch.matmul(torch.from_numpy(x).float(), xi) 
    
    if(x is None):
        lambda_opt = None
    elif(np.all(x == 0)):
        lambda_opt = None
    elif(np.count_nonzero(np.abs(x) > 1e-6) == 2):
        i = np.nonzero(np.abs(x) > 1e-6) # i[0] è la lista [indice1, indice2]
        #print("i:", i)
        diff = i[0][1] - i[0][0]
        passo = v[1] - v[0] # Nel caso semplice sarebbe 1/C
        lambda_opt = round(float(-1 / (diff * passo) * (xi[i[0][1]] - xi[i[0][0]])), 2)
    elif(np.count_nonzero(np.abs(x) > 1e-6) == 1):
        i = np.nonzero(np.abs(x) > 1e-6)
        #print("x:", x)
        #print("i:", i)
        lambda_opt = round(-float(xi[i[0][0]] / v[i[0][0]]), 2) # i[0][0] è l'indice
    else:
        print("A"*10000000) # Caso da trattare
    
    return x, lambda_opt, objective_values, x_plus

# Dimension of vectors xi, v, and x
C = 17
M = 1

np.random.seed(43)  
torch.manual_seed(43)
conta_istanze_risolubili = 0
SUCCESS = 1

for i in range(1000):
    xi = torch.sort(torch.rand(C))[0]
    w0 = 0.3
    r = 0.43
    min_w, max_w = w0 - r, w0 + r
    v = torch.linspace(min_w, max_w - (max_w - min_w)/C, steps=C)
    w = torch.rand(M)
    #w = v[torch.randint(0, C, (1,))]  
    
    #print(f"\nIteration: {i}")
    #print("DATI:")
    #print("xi:", np.array2string(np.array(xi), precision=2, separator=', '))
    #print("v:", np.array2string(np.array(v), precision=2, separator=', '))
    #print("w:", w, "\n")
    
    x_opt, lambda_opt, phi_opt, x_plus = knapsack_specialized_pruning_complete(xi, v, w, C)
    
    # cvxpy solution with inequality constraint (pruning)
    x_pruning = cp.Variable(C)
    equality_constraint_pruning = v @ x_pruning == w
    constraints_pruning = [
        equality_constraint_pruning,
        cp.sum(x_pruning) <= 1,
        x_pruning >= 0,
        x_pruning <= 1
    ]
    objective_pruning = cp.Minimize(xi @ x_pruning)
    prob_pruning = cp.Problem(objective_pruning, constraints_pruning)
    prob_pruning.solve(solver=cp.ECOS)

    #if prob_pruning.value is not None and x_pruning.value is not None:
    #    print(f"Objective value (pruning): {prob_pruning.value:.2f}")
    #    print("Optimal x (pruning):", 
    #          np.array2string(x_pruning.value, precision=2, separator=', ', suppress_small=True))
    #else:
    #    print("No solution found for the problem with pruning.")
    
    #print("x1:", x_pruning.value)
    #print("x2:", x_opt)

    if(lambda_opt is not None):
        conta_istanze_risolubili += 1
        if(not np.allclose(x_pruning.value, x_opt, rtol=1e-5, atol=1e-2)):
            SUCCESS = 0
            print(f"{i} ERRORE NELLE SOLUZIONI:")
            print("xi:", np.array2string(np.array(xi), precision=2, separator=', '))
            print("v:", np.array2string(np.array(v), precision=2, separator=', '))
            print("w:", round(float(w), 4), "\n")
            print("x_opt:", x_opt)
            print("x_pruning.value:", x_pruning.value)
            print("xi/v:", xi/v)
            print("x_plus:", x_plus)
            #print("Soluzione del solver:", x1)
            #print("Soluzione mia:", x2)
            #print("f.o. solver:", round(prob_pruning.value, 4))
            #print("f.o. mia:", round(float(phi_opt), 4))
            #print(f"Il vincolo w=v*x è soddisfatto con la mia soluzione? w = {w}, v*x={v @ x_opt}")
            print("-"*40)
        if(np.abs(equality_constraint_pruning.dual_value[0] - lambda_opt) > 5e-2):
            SUCCESS = 0
            print(f"{i} ERRORE NEI MOLTIPLICATORI:")
            print("xi:", np.array2string(np.array(xi), precision=2, separator=', '))
            print("v:", np.array2string(np.array(v), precision=2, separator=', '))
            print("w:", round(float(w), 4), "\n")
            print("x_opt:", x_opt)
            print("x_pruning.value:", x_pruning.value)
            print("xi/v:", xi/v)
            print("x_plus:", x_plus)
            #print("Soluzione del solver:", x1)
            #print("Soluzione mia:", x2)
            #print("f.o. solver:", round(prob_pruning.value, 4))
            #print("f.o. mia:", round(float(phi_opt), 4))
            #print(f"Il vincolo w=v*x è soddisfatto con la mia soluzione? w = {w}, v*x={v @ x_opt}")
            print("y1:", equality_constraint_pruning.dual_value[:])
            print("y2:", lambda_opt)
            print("-"*40)
            
    #else:
    #    print("Objective value (knapsack_specialized_pruning):", phi_opt)
    #    print("Optimal x (knapsack_specialized_pruning):", x_opt)
    
    #print("-"*40)
if(SUCCESS == 1):
    print("TEST CONCLUSO CON SUCCESSO!")
    
print("\n\nconta_istanze_risolubili:", conta_istanze_risolubili)

TEST CONCLUSO CON SUCCESSO!


conta_istanze_risolubili: 710


# TEST CON w generico

### Effettuo anche un paio di conteggi e li stampo...

In [8]:
def knapsack_specialized_pruning_complete(xi, v, w, C):
    #print("************** DENTRO knapsack_specialized_pruning_complete **************")
    b_list = []
    b = 0

    # Compute breakpoint vector x_plus
    while True:
        delta_xi = (xi[b + 1:] - xi[b])
        delta_v = (v[b + 1:] - v[b])
        b = torch.argmin(delta_xi / delta_v) + 1 + b_list[-1] if b_list else 0

        if b != C - 1:
            b_list.append(int(b))

        if b + 1 > C - 1:
            break
    b_list.append(C - 1)
    x_plus = torch.zeros(C, dtype=torch.int32)
    b_tensor = torch.tensor(b_list, dtype=torch.int32)
    x_plus[b_tensor] = 1
    #print("xi:", xi)
    #print("x_plus:", x_plus)
    if(w > v[-1] or w < v[0]):
        #print("No solutions.")
        x = None
        objective_values = None
    else:
        #b_vector = np.argsort(xi/v)
        ratio = (xi/v).numpy()
        neg_indices = np.where(ratio < 0)[0]
        neg_sorted = neg_indices[np.argsort(ratio[neg_indices])[::-1]]
        pos_indices = np.where(ratio >= 0)[0]
        pos_sorted = pos_indices[np.argsort(ratio[pos_indices])]
        b_vector = np.concatenate([neg_sorted, pos_sorted])
        
        #print("b_vector:", b_vector)
        #print("w/v[b_vector[0]]:", w/v[b_vector[0]])

        b = 0
        while(w/v[b_vector[b]] < 0 or x_plus[b_vector[b]] == 0):
            b += 1
            #print(f"w/v[b_vector[{b}]]:", w/v[b_vector[b]])

        b_list_new = [b_vector[b]] 
        #print("b è stato inizializzato a:", b)
        #print("b_list_new:", b_list_new)
        
        #print(f"Inizializzazione di b_vector[b]: {b_vector[b]}")
        #print(f"w/v[b_vector[b]] > 1: {w/v[b_vector[b]] > 1}, x_plus[b_vector[b]] == 0: {x_plus[b_vector[b]] == 0}, int(b_vector[b]) < int(b_list_new[0]): {int(b_vector[b]) < int(b_list_new[0])}")
        while(w/v[b_vector[b]] > 1 or x_plus[b_vector[b]] == 0 or int(b_vector[b]) < int(b_list_new[0])):
            b += 1
            if(w/v[b_vector[b]] < 0):
                continue
            if(x_plus[b_vector[b]] == 1 and int(b_vector[b]) > int(b_list_new[0])):
                b_list_new.append(b_vector[b])
                #print(f"w: {w}, v[b_vector[b]]: {v[b_vector[b]]}, w/v[b_vector[b]]: {w/v[b_vector[b]]}")
                #print(f"b_list_new:", b_list_new)
                #print("xi (sicurezza):", xi)
            if(len(b_list_new) > 2):
                b_list_new.pop(0)
                
        if(len(b_list_new) == 1 and w/v[b_list_new[0]] > 1):
            b_list_new[0] = 0
            
        #print(f"b_list_new:", b_list_new)
        x = np.zeros(C)
        
        if(len(b_list_new) == 1):
            x[b_list_new[0]] = round(float(w/v[b_list_new[0]]), 2)
        else:
            x1, x2 = torch.zeros(2, len(w), C, dtype=torch.float32)
            x1[torch.arange(len(w)), b_list_new[0]] = 1
            x2[torch.arange(len(w)), b_list_new[1]] = 1
            numerator = w - torch.matmul(x2, v)
            denominator = torch.matmul((x1 - x2), v)
            theta = numerator / denominator
            x[b_list_new[0]] = round(float(theta), 5)
            x[b_list_new[1]] = round(float(1 - theta), 5)
        objective_values = torch.matmul(torch.from_numpy(x).float(), xi) 
    
    if(x is None):
        lambda_opt = None
    elif(np.all(x == 0)):
        lambda_opt = None
    elif(np.count_nonzero(np.abs(x) > 1e-6) == 2):
        i = np.nonzero(np.abs(x) > 1e-6) # i[0] è la lista [indice1, indice2]
        #print("i:", i)
        diff = i[0][1] - i[0][0]
        passo = v[1] - v[0] # Nel caso semplice sarebbe 1/C
        lambda_opt = round(float(-1 / (diff * passo) * (xi[i[0][1]] - xi[i[0][0]])), 2)
    elif(np.count_nonzero(np.abs(x) > 1e-6) == 1):
        i = np.nonzero(np.abs(x) > 1e-6)
        #print("x:", x)
        #print("i:", i)
        lambda_opt = round(-float(xi[i[0][0]] / v[i[0][0]]), 2) # i[0][0] è l'indice
    else:
        print("ERROR: SOLUTION NOT ACCEPTABLE!")

    #print("**************************************************************************")
    return x, lambda_opt, objective_values, x_plus

# Dimension of vectors xi, v, and x
C = 29
M = 1

np.random.seed(42)  
torch.manual_seed(42)

conta_istanze_risolubili = 0
conta_soluzioni_sotto_soglia = 0
conta_moltiplicatori_errati = 0
conta_soluzioni_errate = 0

SUCCESS = 1

for i in range(1000):
    xi = torch.sort(torch.rand(C))[0]
    w0 = 0
    r = 1.1
    min_w, max_w = w0 - r, w0 + r
    v = torch.linspace(min_w, max_w - (max_w - min_w)/C, steps=C)
    a = -1
    b = 1
    w = torch.rand(M) * (b - a) + a


    #w = v[torch.randint(0, C, (1,))]  
    
    #print(f"\nIteration: {i}")
    #print("DATI:")
    #print("xi:", np.array2string(np.array(xi), precision=2, separator=', '))
    #print("v:", np.array2string(np.array(v), precision=2, separator=', '))
    #print("w:", w, "\n")
    
    x_opt, lambda_opt, phi_opt, x_plus = knapsack_specialized_pruning_complete(xi, v, w, C)
    
    soglia = 0.1
    if(x_opt is not None):
        if(np.max(x_opt) < soglia):
            conta_soluzioni_sotto_soglia += 1
            #print("x_opt:", x_opt, "lambda_opt:", lambda_opt)
            
    # cvxpy solution with inequality constraint (pruning)
    x_pruning = cp.Variable(C)
    equality_constraint_pruning = v @ x_pruning == w
    constraints_pruning = [
        equality_constraint_pruning,
        cp.sum(x_pruning) <= 1,
        x_pruning >= 0,
        x_pruning <= 1
    ]
    objective_pruning = cp.Minimize(xi @ x_pruning)
    prob_pruning = cp.Problem(objective_pruning, constraints_pruning)
    prob_pruning.solve(solver=cp.ECOS)

    #if prob_pruning.value is not None and x_pruning.value is not None:
    #    print(f"Objective value (pruning): {prob_pruning.value:.2f}")
    #    print("Optimal x (pruning):", 
    #          np.array2string(x_pruning.value, precision=2, separator=', ', suppress_small=True))
    #else:
    #    print("No solution found for the problem with pruning.")
    
    #print("x1:", x_pruning.value)
    #print("x2:", x_opt)

    if(lambda_opt is not None):
        conta_istanze_risolubili += 1
        if(not np.allclose(x_pruning.value, x_opt, rtol=1e-5, atol=1e-2)):
            conta_soluzioni_errate += 1
            SUCCESS = 0
            print(f"{i} ERRORE NELLE SOLUZIONI:")
            print("xi:", np.array2string(np.array(xi), precision=2, separator=', '))
            print("v:", np.array2string(np.array(v), precision=2, separator=', '))
            print("w:", round(float(w), 4), "\n")
            print("x_opt:", x_opt)
            print("x_pruning.value:", x_pruning.value)
            print("xi/v:", xi/v)
            print("x_plus:", x_plus)
            #print("Soluzione del solver:", x1)
            #print("Soluzione mia:", x2)
            #print("f.o. solver:", round(prob_pruning.value, 4))
            #print("f.o. mia:", round(float(phi_opt), 4))
            #print(f"Il vincolo w=v*x è soddisfatto con la mia soluzione? w = {w}, v*x={v @ x_opt}")
            print("-"*40)
        if(np.abs(equality_constraint_pruning.dual_value[0] - lambda_opt) > 5e-2):
            conta_moltiplicatori_errati += 1
            SUCCESS = 0
            print(f"{i} ERRORE NEI MOLTIPLICATORI:")
            print("xi:", np.array2string(np.array(xi), precision=2, separator=', '))
            print("v:", np.array2string(np.array(v), precision=2, separator=', '))
            print("w:", round(float(w), 4), "\n")
            print("x_opt:", x_opt)
            print("x_pruning.value:", x_pruning.value)
            print("xi/v:", xi/v)
            print("x_plus:", x_plus)
            #print("Soluzione del solver:", x1)
            #print("Soluzione mia:", x2)
            #print("f.o. solver:", round(prob_pruning.value, 4))
            #print("f.o. mia:", round(float(phi_opt), 4))
            #print(f"Il vincolo w=v*x è soddisfatto con la mia soluzione? w = {w}, v*x={v @ x_opt}")
            print("y1:", equality_constraint_pruning.dual_value[:])
            print("y2:", lambda_opt)
            print("-"*40)
            print("\n\n")
            
    #else:
    #    print("Objective value (knapsack_specialized_pruning):", phi_opt)
    #    print("Optimal x (knapsack_specialized_pruning):", x_opt)
    
    #print("-"*40)
if(SUCCESS == 1):
    print("\nTEST CONCLUSO CON SUCCESSO!")
    
print("\n\nconta_istanze_risolubili:", conta_istanze_risolubili)
print("conta_soluzioni_sotto_soglia:", conta_soluzioni_sotto_soglia)
print("conta_soluzioni_errate:", conta_soluzioni_errate)
print("conta_moltiplicatori_errati:", conta_moltiplicatori_errati)


TEST CONCLUSO CON SUCCESSO!


conta_istanze_risolubili: 997
conta_soluzioni_sotto_soglia: 86
conta_soluzioni_errate: 0
conta_moltiplicatori_errati: 0


# VERSIONE PULITA SENZA DEBUG

In [9]:
def knapsack_specialized_pruning_complete(xi, v, w, C):

    b_list = []
    b = 0

    # Compute breakpoint vector x_plus
    while True:
        delta_xi = (xi[b + 1:] - xi[b])
        delta_v = (v[b + 1:] - v[b])
        b = torch.argmin(delta_xi / delta_v) + 1 + b_list[-1] if b_list else 0

        if b != C - 1:
            b_list.append(int(b))

        if b + 1 > C - 1:
            break
            
    b_list.append(C - 1)
    x_plus = torch.zeros(C, dtype=torch.int32)
    b_tensor = torch.tensor(b_list, dtype=torch.int32)
    x_plus[b_tensor] = 1

    if(w > v[-1] or w < v[0]):
        x = None
        objective_values = None
    else:
        ratio = (xi/v).numpy()
        neg_indices = np.where(ratio < 0)[0]
        neg_sorted = neg_indices[np.argsort(ratio[neg_indices])[::-1]]
        pos_indices = np.where(ratio >= 0)[0]
        pos_sorted = pos_indices[np.argsort(ratio[pos_indices])]
        b_vector = np.concatenate([neg_sorted, pos_sorted])

        b = 0
        while(w/v[b_vector[b]] < 0 or x_plus[b_vector[b]] == 0):
            b += 1

        b_list_new = [b_vector[b]] 
        
        while(w/v[b_vector[b]] > 1 or x_plus[b_vector[b]] == 0 or int(b_vector[b]) < int(b_list_new[0])):
            b += 1
            if(w/v[b_vector[b]] < 0):
                continue
            if(x_plus[b_vector[b]] == 1 and int(b_vector[b]) > int(b_list_new[0])):
                b_list_new.append(b_vector[b])
            if(len(b_list_new) > 2):
                b_list_new.pop(0)
                
        if(len(b_list_new) == 1 and w/v[b_list_new[0]] > 1):
            b_list_new[0] = 0
            
        x = np.zeros(C)
        
        if(len(b_list_new) == 1):
            x[b_list_new[0]] = round(float(w/v[b_list_new[0]]), 5)
        else:
            x1, x2 = torch.zeros(2, len(w), C, dtype=torch.float32)
            x1[torch.arange(len(w)), b_list_new[0]] = 1
            x2[torch.arange(len(w)), b_list_new[1]] = 1
            numerator = w - torch.matmul(x2, v)
            denominator = torch.matmul((x1 - x2), v)
            theta = numerator / denominator
            x[b_list_new[0]] = round(float(theta), 5)
            x[b_list_new[1]] = round(float(1 - theta), 5)
        objective_values = torch.matmul(torch.from_numpy(x).float(), xi) 
    
    if(x is None): # Al caso di soluzione vuota faccio corrispondere un moltiplicatore vuoto 
        lambda_opt = None
    elif(np.all(x == 0)): # Al caso di soluzione nulla faccio corrispondere un moltiplicatore nullo
        lambda_opt = 0
    elif(np.count_nonzero(np.abs(x) > 1e-6) == 2):
        i = np.nonzero(np.abs(x) > 1e-6) # i[0] è la lista [indice1, indice2]
        diff = i[0][1] - i[0][0]
        passo = v[1] - v[0] # Nel caso semplice sarebbe 1/C
        lambda_opt = round(float(-1 / (diff * passo) * (xi[i[0][1]] - xi[i[0][0]])), 5)
    elif(np.count_nonzero(np.abs(x) > 1e-6) == 1):
        i = np.nonzero(np.abs(x) > 1e-6)
        lambda_opt = round(-float(xi[i[0][0]] / v[i[0][0]]), 5) # i[0][0] è l'indice
    else:
        print("ERROR: SOLUTION NOT ACCEPTABLE!")

    return x, lambda_opt, objective_values, x_plus

# Dimension of vectors xi, v, and x
C = 29
M = 1

np.random.seed(42)  
torch.manual_seed(42)

conta_istanze_risolubili = 0
conta_soluzioni_sotto_soglia = 0
conta_moltiplicatori_errati = 0
conta_soluzioni_errate = 0

SUCCESS = 1

for i in range(1000):
    xi = torch.sort(torch.rand(C))[0]
    w0 = -0.01
    r = 1.32
    min_w, max_w = w0 - r, w0 + r
    v = torch.linspace(min_w, max_w - (max_w - min_w)/C, steps=C)
    a = -0.23
    b = 2.11
    w = torch.rand(M) * (b - a) + a
    
    x_opt, lambda_opt, phi_opt, x_plus = knapsack_specialized_pruning_complete(xi, v, w, C)
    
    soglia = 0.1
    if(x_opt is not None):
        conta_istanze_risolubili += 1
        if(np.max(x_opt) < soglia):
            conta_soluzioni_sotto_soglia += 1
            
    # cvxpy solution with inequality constraint (pruning)
    x_pruning = cp.Variable(C)
    equality_constraint_pruning = v @ x_pruning == w
    constraints_pruning = [
        equality_constraint_pruning,
        cp.sum(x_pruning) <= 1,
        x_pruning >= 0,
        x_pruning <= 1
    ]
    objective_pruning = cp.Minimize(xi @ x_pruning)
    prob_pruning = cp.Problem(objective_pruning, constraints_pruning)
    prob_pruning.solve(solver=cp.ECOS)
    
    if(x_opt is not None):
        if(not np.allclose(x_pruning.value, x_opt, rtol=1e-5, atol=1e-2)):
            conta_soluzioni_errate += 1
            SUCCESS = 0
            print(f"{i} ERRORE NELLE SOLUZIONI:")
        if(np.abs(equality_constraint_pruning.dual_value[0] - lambda_opt) > 5e-2):
            conta_moltiplicatori_errati += 1
            SUCCESS = 0
            print(f"{i} ERRORE NEI MOLTIPLICATORI:")
            print(x_pruning.value)
            
if(SUCCESS == 1):
    print("\nTEST CONCLUSO CON SUCCESSO!")
    
print("\n\nconta_istanze_risolubili:", conta_istanze_risolubili)
print("conta_soluzioni_sotto_soglia:", conta_soluzioni_sotto_soglia)
print("conta_soluzioni_errate:", conta_soluzioni_errate)
print("conta_moltiplicatori_errati:", conta_moltiplicatori_errati)


TEST CONCLUSO CON SUCCESSO!


conta_istanze_risolubili: 600
conta_soluzioni_sotto_soglia: 106
conta_soluzioni_errate: 0
conta_moltiplicatori_errati: 0


# CASO w > v[-1] or w < v[0] (NON PUÒ ESISTERE LA SOLUZIONE MA IN QUALCHE MODO BISOGNA DARLA)

In [10]:
def knapsack_specialized_pruning_complete(xi, v, w, C):

    b_list = []
    b = 0

    # Compute breakpoint vector x_plus
    while True:
        delta_xi = (xi[b + 1:] - xi[b])
        delta_v = (v[b + 1:] - v[b])
        b = torch.argmin(delta_xi / delta_v) + 1 + b_list[-1] if b_list else 0
        if b != C - 1:
            b_list.append(int(b))
        if b + 1 > C - 1:
            break
            
    b_list.append(C - 1)
    x_plus = torch.zeros(C, dtype=torch.int32)
    b_tensor = torch.tensor(b_list, dtype=torch.int32)
    x_plus[b_tensor] = 1

    if(w > v[-1]):
        x = np.zeros(C)
        x[-1] = 1
    elif(w < v[0]):
        x = np.zeros(C)
        x[0] = 1
    else:
        ratio = (xi/v).numpy()
        neg_indices = np.where(ratio < 0)[0]
        neg_sorted = neg_indices[np.argsort(ratio[neg_indices])[::-1]]
        pos_indices = np.where(ratio >= 0)[0]
        pos_sorted = pos_indices[np.argsort(ratio[pos_indices])]
        b_vector = np.concatenate([neg_sorted, pos_sorted])

        b = 0
        while(w/v[b_vector[b]] < 0 or x_plus[b_vector[b]] == 0):
            b += 1

        b_list_new = [b_vector[b]] 
        
        while(w/v[b_vector[b]] > 1 or x_plus[b_vector[b]] == 0 or int(b_vector[b]) < int(b_list_new[0])):
            b += 1
            if(w/v[b_vector[b]] < 0):
                continue
            if(x_plus[b_vector[b]] == 1 and int(b_vector[b]) > int(b_list_new[0])):
                b_list_new.append(b_vector[b])
            if(len(b_list_new) > 2):
                b_list_new.pop(0)
                
        if(len(b_list_new) == 1 and w/v[b_list_new[0]] > 1):
            b_list_new[0] = 0
            
        x = np.zeros(C)
        
        if(len(b_list_new) == 1):
            x[b_list_new[0]] = round(float(w/v[b_list_new[0]]), 5)
        else:
            x1, x2 = torch.zeros(2, len(w), C, dtype=torch.float32)
            x1[torch.arange(len(w)), b_list_new[0]] = 1
            x2[torch.arange(len(w)), b_list_new[1]] = 1
            numerator = w - torch.matmul(x2, v)
            denominator = torch.matmul((x1 - x2), v)
            theta = numerator / denominator
            x[b_list_new[0]] = round(float(theta), 5)
            x[b_list_new[1]] = round(float(1 - theta), 5)
            
        objective_values = torch.matmul(torch.from_numpy(x).float(), xi) 
    
    if(x is None): # Al caso di soluzione vuota faccio corrispondere un moltiplicatore vuoto 
        lambda_opt = None
    elif(np.all(x == 0)): # Al caso di soluzione nulla faccio corrispondere un moltiplicatore nullo
        lambda_opt = 0
    elif(np.count_nonzero(np.abs(x) > 1e-6) == 2):
        i = np.nonzero(np.abs(x) > 1e-6) # i[0] è la lista [indice1, indice2]
        diff = i[0][1] - i[0][0]
        passo = v[1] - v[0] # Nel caso semplice sarebbe 1/C
        lambda_opt = round(float(-1 / (diff * passo) * (xi[i[0][1]] - xi[i[0][0]])), 5)
    elif(np.count_nonzero(np.abs(x) > 1e-6) == 1):
        i = np.nonzero(np.abs(x) > 1e-6)
        lambda_opt = round(-float(xi[i[0][0]] / v[i[0][0]]), 5) # i[0][0] è l'indice
    else:
        print("ERROR: SOLUTION NOT ACCEPTABLE!")

    objective_values = xi @ x
    return x, lambda_opt, objective_values, x_plus

# Dimension of vectors xi, v, and x
C = 29
M = 1

np.random.seed(41)  
torch.manual_seed(41)

conta_istanze_risolubili = 0
conta_soluzioni_sotto_soglia = 0
conta_moltiplicatori_errati = 0
conta_soluzioni_errate = 0

SUCCESS = 1

for i in range(1000):
    xi = torch.sort(torch.rand(C))[0]
    w0 = -0.01
    r = 1.32
    min_w, max_w = w0 - r, w0 + r
    v = torch.linspace(min_w, max_w - (max_w - min_w)/C, steps=C)
    a = -2
    b = 2.2
    w = torch.rand(M) * (b - a) + a
    
    x_opt, lambda_opt, phi_opt, x_plus = knapsack_specialized_pruning_complete(xi, v, w, C)
            
    # cvxpy solution with inequality constraint (pruning)
    x_pruning = cp.Variable(C)
    equality_constraint_pruning = v @ x_pruning == w
    constraints_pruning = [
        equality_constraint_pruning,
        cp.sum(x_pruning) <= 1,
        x_pruning >= 0,
        x_pruning <= 1
    ]
    objective_pruning = cp.Minimize(xi @ x_pruning)
    prob_pruning = cp.Problem(objective_pruning, constraints_pruning)
    prob_pruning.solve(solver=cp.ECOS)
    
    soglia = 0.0001
    if(x_pruning.value is not None):
        conta_istanze_risolubili += 1
        if(np.max(x_opt) < soglia):
            conta_soluzioni_sotto_soglia += 1
    
    if(x_pruning.value is not None):
        if(not np.allclose(x_pruning.value, x_opt, rtol=1e-5, atol=1e-2)):
            conta_soluzioni_errate += 1
            SUCCESS = 0
            print(f"{i} ERRORE NELLE SOLUZIONI:")
        if(np.abs(equality_constraint_pruning.dual_value[0] - lambda_opt) > 5e-2):
            conta_moltiplicatori_errati += 1
            SUCCESS = 0
            print(f"{i} ERRORE NEI MOLTIPLICATORI:")
            print(x_pruning.value)
    else: # da eliminare una volta verificato che i casi di soluzione non ammissibile sono gestiti correttamente
        print("x_opt:", x_opt)
        print("x_pruning.value:", x_pruning.value)
        print("lambda_opt:", lambda_opt)
        print("equality_constraint_pruning.dual_value:", equality_constraint_pruning.dual_value)
        print("-"*50)
            
if(SUCCESS == 1):
    print("\nTEST CONCLUSO CON SUCCESSO!")
    
print("\n\nconta_istanze_risolubili:", conta_istanze_risolubili)
print("conta_soluzioni_sotto_soglia:", conta_soluzioni_sotto_soglia)
print("conta_soluzioni_errate:", conta_soluzioni_errate)
print("conta_moltiplicatori_errati:", conta_moltiplicatori_errati)

x_opt: [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0.]
x_pruning.value: None
lambda_opt: 0.0063
equality_constraint_pruning.dual_value: None
--------------------------------------------------
x_opt: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 1.]
x_pruning.value: None
lambda_opt: -0.81678
equality_constraint_pruning.dual_value: None
--------------------------------------------------
x_opt: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 1.]
x_pruning.value: None
lambda_opt: -0.80594
equality_constraint_pruning.dual_value: None
--------------------------------------------------
x_opt: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 1.]
x_pruning.value: None
lambda_opt: -0.81024
equality_constraint_pruning.dual_value: None
--------------------------------------------------
x_opt: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.

x_opt: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 1.]
x_pruning.value: None
lambda_opt: -0.81666
equality_constraint_pruning.dual_value: None
--------------------------------------------------
x_opt: [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0.]
x_pruning.value: None
lambda_opt: 0.01361
equality_constraint_pruning.dual_value: None
--------------------------------------------------
x_opt: [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0.]
x_pruning.value: None
lambda_opt: 0.0157
equality_constraint_pruning.dual_value: None
--------------------------------------------------
x_opt: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 1.]
x_pruning.value: None
lambda_opt: -0.81623
equality_constraint_pruning.dual_value: None
--------------------------------------------------
x_opt: [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 

x_opt: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 1.]
x_pruning.value: None
lambda_opt: -0.81394
equality_constraint_pruning.dual_value: None
--------------------------------------------------
x_opt: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 1.]
x_pruning.value: None
lambda_opt: -0.79412
equality_constraint_pruning.dual_value: None
--------------------------------------------------
x_opt: [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0.]
x_pruning.value: None
lambda_opt: 0.01009
equality_constraint_pruning.dual_value: None
--------------------------------------------------
x_opt: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 1.]
x_pruning.value: None
lambda_opt: -0.82004
equality_constraint_pruning.dual_value: None
--------------------------------------------------
x_opt: [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0

x_opt: [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0.]
x_pruning.value: None
lambda_opt: 0.02232
equality_constraint_pruning.dual_value: None
--------------------------------------------------
x_opt: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 1.]
x_pruning.value: None
lambda_opt: -0.81072
equality_constraint_pruning.dual_value: None
--------------------------------------------------
x_opt: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 1.]
x_pruning.value: None
lambda_opt: -0.76439
equality_constraint_pruning.dual_value: None
--------------------------------------------------
x_opt: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 1.]
x_pruning.value: None
lambda_opt: -0.79589
equality_constraint_pruning.dual_value: None
--------------------------------------------------
x_opt: [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0

x_opt: [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0.]
x_pruning.value: None
lambda_opt: 0.00807
equality_constraint_pruning.dual_value: None
--------------------------------------------------
x_opt: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 1.]
x_pruning.value: None
lambda_opt: -0.80467
equality_constraint_pruning.dual_value: None
--------------------------------------------------
x_opt: [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0.]
x_pruning.value: None
lambda_opt: 0.00228
equality_constraint_pruning.dual_value: None
--------------------------------------------------
x_opt: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 1.]
x_pruning.value: None
lambda_opt: -0.78753
equality_constraint_pruning.dual_value: None
--------------------------------------------------
x_opt: [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.

x_opt: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 1.]
x_pruning.value: None
lambda_opt: -0.80792
equality_constraint_pruning.dual_value: None
--------------------------------------------------
x_opt: [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0.]
x_pruning.value: None
lambda_opt: 0.04956
equality_constraint_pruning.dual_value: None
--------------------------------------------------
x_opt: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 1.]
x_pruning.value: None
lambda_opt: -0.7842
equality_constraint_pruning.dual_value: None
--------------------------------------------------
x_opt: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 1.]
x_pruning.value: None
lambda_opt: -0.81924
equality_constraint_pruning.dual_value: None
--------------------------------------------------
x_opt: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.

x_opt: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 1.]
x_pruning.value: None
lambda_opt: -0.79904
equality_constraint_pruning.dual_value: None
--------------------------------------------------
x_opt: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 1.]
x_pruning.value: None
lambda_opt: -0.80259
equality_constraint_pruning.dual_value: None
--------------------------------------------------
x_opt: [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0.]
x_pruning.value: None
lambda_opt: 0.01716
equality_constraint_pruning.dual_value: None
--------------------------------------------------
x_opt: [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0.]
x_pruning.value: None
lambda_opt: 0.03969
equality_constraint_pruning.dual_value: None
--------------------------------------------------
x_opt: [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.

x_opt: [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0.]
x_pruning.value: None
lambda_opt: 0.00647
equality_constraint_pruning.dual_value: None
--------------------------------------------------
x_opt: [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0.]
x_pruning.value: None
lambda_opt: 0.07383
equality_constraint_pruning.dual_value: None
--------------------------------------------------
x_opt: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 1.]
x_pruning.value: None
lambda_opt: -0.8181
equality_constraint_pruning.dual_value: None
--------------------------------------------------
x_opt: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 1.]
x_pruning.value: None
lambda_opt: -0.81779
equality_constraint_pruning.dual_value: None
--------------------------------------------------
x_opt: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 

x_opt: [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0.]
x_pruning.value: None
lambda_opt: 0.02403
equality_constraint_pruning.dual_value: None
--------------------------------------------------
x_opt: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 1.]
x_pruning.value: None
lambda_opt: -0.81913
equality_constraint_pruning.dual_value: None
--------------------------------------------------
x_opt: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 1.]
x_pruning.value: None
lambda_opt: -0.78947
equality_constraint_pruning.dual_value: None
--------------------------------------------------
x_opt: [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0.]
x_pruning.value: None
lambda_opt: 0.04461
equality_constraint_pruning.dual_value: None
--------------------------------------------------
x_opt: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.

x_opt: [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0.]
x_pruning.value: None
lambda_opt: 0.01482
equality_constraint_pruning.dual_value: None
--------------------------------------------------
x_opt: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 1.]
x_pruning.value: None
lambda_opt: -0.78503
equality_constraint_pruning.dual_value: None
--------------------------------------------------
x_opt: [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0.]
x_pruning.value: None
lambda_opt: 0.09528
equality_constraint_pruning.dual_value: None
--------------------------------------------------
x_opt: [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0.]
x_pruning.value: None
lambda_opt: 0.00962
equality_constraint_pruning.dual_value: None
--------------------------------------------------
x_opt: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 

# TEST CON PARALLELIZZAZIONE (vedi notebook PruningVectorized.ipynb)